In [12]:
from dotenv import load_dotenv

load_dotenv()

True

## Local MCP server

In [13]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
            "transport": "stdio",
            "command": "python",
            "args": ["resources/2.1_mcp_server.py"]
        }
    }
)

https://modelcontextprotocol.io/specification/2025-11-25/basic/transports

In [14]:
# Get tools
tools = await client.get_tools()

# Get resources
resources = await client.get_resources("local_server")

# Get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [15]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

In [16]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [17]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='235358dd-b507-463b-87fc-8deabf518a8e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 270, 'total_tokens': 427, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csc4nkKuCEVOQFYWTqJVx2UixT1GG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7148-296a-7593-b551-09d8f23f9d4c-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters library'}, 'id': 'call_GA9NVrW6y7uaBMP6oFx0aQsX', 'type': 'tool_call'}], usage_metadata={'

## Online MCP

https://mcp.so/servers

MCP Server used below:

https://mcp.so/server/time/modelcontextprotocol

In [18]:
import sys

client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": sys.executable, # uses the same Python as this notebook kernel
            "args": [
                "-m",
                "mcp_server_time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [19]:
tools

[StructuredTool(name='get_current_time', description='Get current time in a specific timezones', args_schema={'type': 'object', 'properties': {'timezone': {'type': 'string', 'description': "IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'America/New_York' as local timezone if no timezone provided by the user."}}, 'required': ['timezone']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11e23cae0>),
 StructuredTool(name='convert_time', description='Convert time between timezones', args_schema={'type': 'object', 'properties': {'source_timezone': {'type': 'string', 'description': "Source IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'America/New_York' as local timezone if no source timezone provided by the user."}, 'time': {'type': 'string', 'description': 'Time to convert in 24-hour format (HH:MM)'}, 'target_timezone': {'type': 'string', 'description': "Target IANA timezone 

In [20]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools
)

In [21]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='a92d1185-7e51-4e56-bf05-0e75fd8617e2'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 283, 'prompt_tokens': 296, 'total_tokens': 579, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csc59peWaFLdc8rnXracYugjh7AsA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7148-7f9f-78d2-aa23-5565372eaf29-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': 'call_IYcYoJz29Ua2LepCvzOldfVH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 296, 'output_tokens': 